In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from time import sleep
from time import time
import random

In [2]:
# random_sleep = round(random.uniform(2, 3), 2)
def scraping_airbnb(url):
    
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-search-engine-choice-screen")
    options.add_argument("--headless")
    options.add_argument('--lang=en')
    options.add_argument('--disable-geolocation')
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    # Esperar que la página cargue y gestionar popup de traducción
    try:
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@class='c1lbtiq8 atm_mk_stnw88 atm_9s_1txwivl atm_fq_1tcgj5g atm_wq_kb7nvz atm_tk_1tcgj5g dir dir-ltr']"))
        ).click()
    except:
        pass
    random_sleep = round(random.uniform(2, 3), 2)
    sleep(random_sleep)
    price = extract_price(driver)
    sleep(random_sleep)
    title = extract_title(driver)
    sleep(random_sleep)
    rating = extract_rating(driver)
    sleep(random_sleep)
    number_reviews = extract_number_reviews(driver)
    sleep(random_sleep)
    guest_favorite = extract_guest_favorite(driver)
    sleep(random_sleep)
    type_host = extract_type_host(driver)
    sleep(random_sleep)
    data_list, hosting_time = extract_data_list(driver)
    sleep(random_sleep)
    all_reviews = extract_reviews(driver, number_reviews)
    driver.quit()
    return guest_favorite, rating, number_reviews, type_host, hosting_time, all_reviews, title, data_list, price
    
def extract_price(driver):
    # Lista de posibles XPaths para localizar el precio
    xpaths = [
        '//div[@class="l1x1206l atm_7l_jt7fhx atm_r3_1e5hqsa dir dir-ltr"]',
        "//span[contains(text(),'per night')]",
        "//span[contains(text(),'€')]",
        "//div[contains(text(),'€')]",
        '//span[@class="_11jcbg2"]',
        '//div[@class="_1jo4hgw"]',
        '//span[@class="_1qgfaxb1"]',
        '//div[@class="_ati8ih"]',
        '//span[@class="a8jt5op atm_3f_idpfg4 atm_7h_hxbz6r atm_7i_ysn8ba atm_e2_t94yts atm_ks_zryt35 atm_l8_idpfg4 atm_vv_1q9ccgz atm_vy_t94yts aze35hn atm_mk_stnw88 atm_tk_idpfg4 dir dir-ltr"]',
        '//span[@class="a8jt5op atm_3f_idpfg4 atm_7h_hxbz6r atm_7i_ysn8ba atm_e2_t94yts atm_ks_zryt35 atm_l8_idpfg4 atm_vv_1q9ccgz atm_vy_t94yts a1ugchtf atm_mk_stnw88 atm_tk_idpfg4 dir dir-ltr"]'
    ]
    
    # Intentar encontrar el precio usando cada XPath
    for xpath in xpaths:
        try:
            random_sleep = round(random.uniform(2, 3), 2)
            sleep(random_sleep)
            price = driver.find_element(By.XPATH, xpath).text
            # Verificar si el precio es válido (no vacío y no demasiado corto)
            if price and len(price) > 2:
                return price
        except:
            continue
    
    # Si ninguno de los XPaths devuelve un precio válido, retornar NaN
    return np.nan

# def extract_price(driver):
#     try:
#         price = driver.find_element(By.XPATH, '//div[@class="l1x1206l atm_7l_jt7fhx atm_r3_1e5hqsa dir dir-ltr"]').text
#     except:
#         try:
#             price = driver.find_element(By.XPATH, "//span[contains(text(),'per night')]").text
#         except:
#             try:
#                 price = driver.find_element(By.XPATH, '//div[@class="_11jcbg2"]').text
#             except:
#                 try:
#                     price = driver.find_element(By.XPATH, '//div[@class="_ati8ih"]').text
#                 except:
#                     try:
#                         price = driver.find_element(By.XPATH, '//span[@class="a8jt5op atm_3f_idpfg4 atm_7h_hxbz6r atm_7i_ysn8ba atm_e2_t94yts atm_ks_zryt35 atm_l8_idpfg4 atm_vv_1q9ccgz atm_vy_t94yts aze35hn atm_mk_stnw88 atm_tk_idpfg4 dir dir-ltr"]').text
#                     except:
#                         price = np.nan
#         return price
def extract_title(driver):
    try:
        return driver.find_element(By.XPATH, "//div[@class = '_1czgyoo']").text
    except:
        return np.nan
    
def extract_number_reviews(driver):
    try:
        number_reviews = driver.find_element(By.XPATH, '//div[@class="r16onr0j atm_c8_vvn7el atm_g3_k2d186 atm_fr_1vi102y atm_gq_myb0kj atm_vv_qvpr2i atm_c8_sz6sci__14195v1 atm_g3_17zsb9a__14195v1 atm_fr_kzfbxz__14195v1 atm_gq_idpfg4__14195v1 dir dir-ltr"]').text
        return number_reviews
    except:
        try:
            sleep(2)
            number_reviews = driver.find_element(By.XPATH, '//button[@data-testid="pdp-show-all-reviews-button"]').text
            # data-testid="pdp-show-all-reviews-button"
            number_reviews = number_reviews.split(" ")[2]
            return number_reviews
        except:
            try:
                sleep(2)
                number_reviews = driver.find_element(By.XPATH, '//span[@class="ttu4mdj atm_9s_116y0ak dir dir-ltr"]').text
                return number_reviews
            except:
                number_reviews = np.nan
                pass
    try:
        float(number_reviews)
    except:
        pass
    if not isinstance(number_reviews, (int, float, complex)):
        try:
            number_reviews = driver.find_element(By.XPATH, '//span[@class="ttu4mdj atm_9s_116y0ak dir dir-ltr"]').text
            return number_reviews
        except:
            number_reviews = np.nan
    return number_reviews

def extract_guest_favorite(driver):
    guest_favorite = False
    guest = "no"
    try:
        guest = driver.find_element(By.XPATH, '//div[@class="lbjrbi0 atm_le_1y44olf atm_lk_1y44olf atm_ll_1y44olf dir dir-ltr"]').text
    except:
        pass
    if guest != "no":
        guest_favorite = True
    return guest_favorite

def extract_rating(driver):
    try:
        rating = driver.find_element(By.XPATH, '//div[@class="r1lutz1s atm_c8_o7aogt atm_c8_l52nlx__oggzyc dir dir-ltr"]').text
    except:
        try:
            random_sleep = round(random.uniform(2, 3), 2)
            sleep(random_sleep)
            rating = driver.find_element(By.XPATH, '//div[@class="a8jhwcl atm_c8_vvn7el atm_g3_k2d186 atm_fr_1vi102y atm_9s_1txwivl atm_ar_1bp4okc atm_h_1h6ojuz atm_cx_t94yts atm_le_14y27yu atm_c8_sz6sci__14195v1 atm_g3_17zsb9a__14195v1 atm_fr_kzfbxz__14195v1 atm_cx_1l7b3ar__14195v1 atm_le_1l7b3ar__14195v1 dir dir-ltr"]').text
        except:
            rating = np.nan
    return rating

def extract_type_host(driver):
    try:
        type_host = driver.find_element(By.XPATH, '//li[@class="l7n4lsf atm_9s_1o8liyq_keqd55 dir dir-ltr"]').text
    except:
        type_host = np.nan
    return type_host

def extract_data_list(driver):
    try:
        complete_data_list_primer = driver.find_elements(By.XPATH, '//div[@class="o1kjrihn atm_c8_km0zk7 atm_g3_18khvle atm_fr_1m9t47k atm_h3_1y44olf atm_c8_2x1prs__oggzyc atm_g3_1jbyh58__oggzyc atm_fr_11a07z3__oggzyc dir dir-ltr"]')
        complete_data_list = [x.text for x in complete_data_list_primer]
        hosting_time = driver.find_element(By.XPATH, '//div[@class="s1l7gi0l atm_c8_km0zk7 atm_g3_18khvle atm_fr_1m9t47k atm_7l_1esdqks dir dir-ltr"]').text
    except:
        complete_data_list = np.nan
        hosting_time = "New host"
    return complete_data_list, hosting_time

def extract_reviews(driver, number_reviews):
    try:
        show_more_reviews(driver, number_reviews)
        all_reviews = scroll_reviews(driver)
    except:
        try:
            all_reviews = driver.find_elements(By.XPATH, '//div[@class="r1bctolv atm_c8_1sjzizj atm_g3_1dgusqm atm_26_lfmit2_13uojos atm_5j_1y44olf_13uojos atm_l8_1s2714j_13uojos dir dir-ltr"]')
            all_reviews = [x.text for x in all_reviews]
        except:
            all_reviews = np.nan
    return all_reviews

def show_more_reviews(driver, number_reviews):
    try:
        more_reviews_button_xpath = f"//button[contains(text(), 'Show all {number_reviews} reviews')]"
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, more_reviews_button_xpath))
        ).click()
        sleep(2)
    except (TimeoutException, NoSuchElementException):
        pass

def scroll_reviews(driver):
    random_sleep = round(random.uniform(2, 3), 2)
    all_reviews = []

    try:
        review_popup = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "_17itzz4")))
        last_height = driver.execute_script("return arguments[0].scrollHeight", review_popup)
    except:
        return all_reviews

    while len(list(set(all_reviews))) < 100:
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_popup)
        sleep(random_sleep)
        reviews_list = driver.find_elements(By.XPATH, "//div[@class='r1bctolv atm_c8_1sjzizj atm_g3_1dgusqm atm_26_lfmit2_13uojos atm_5j_1y44olf_13uojos atm_l8_1s2714j_13uojos dir dir-ltr']")
        all_reviews.extend([review.text for review in reviews_list if len(review.text) > 3])
        sleep(random_sleep)
        new_height = driver.execute_script("return arguments[0].scrollHeight", review_popup)
        if new_height == last_height:
            break
        last_height = new_height
    return list(set(all_reviews))

In [3]:
def scraping_urls(city_url):
    # WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-search-engine-choice-screen")
    options.add_argument("--headless")
    options.add_argument('--lang=en')
    options.add_argument('--disable-geolocation')
    driver = webdriver.Chrome(options=options)

    # Airbnb Nottingham 
    # url = 'https://www.airbnb.com/s/Nottingham--England--United-Kingdom/homes'
    driver.get(city_url)

    all_urls: list = []
    all_prices: list = []
    # Scraping para la pagina en la que está
    def scrape_urls():
        wait = WebDriverWait(driver, 10)  # Espera hasta 10 segundos
        element = wait.until(EC.presence_of_element_located((By.XPATH, '//a[contains(@href, "/rooms/")]')))
        # sleep(8) # espera a que carge
        listings = driver.find_elements(By.XPATH, '//a[contains(@href, "/rooms/")]')
        urls = [listing.get_attribute('href') for listing in listings]
        sleep(3)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 0.7);")  # Baja al 90% de la página
        sleep(2)
        wait2 = WebDriverWait(driver, 10)
        element2 = wait.until(EC.presence_of_element_located((By.XPATH, '//span[@class="_11jcbg2"]')))
        prices_primer = driver.find_elements(By.XPATH, '//span[@class="_11jcbg2"]')
        # class="_1jo4hgw" class="_1qgfaxb1"
        prices = [price.text for price in prices_primer]
        return urls, prices

    # Loop para todas las paginas
    while True:
        page_urls, price_list = scrape_urls() # Esto funciona porque cada vez que vamos a una pagina nueva la url principal no cambia
        all_urls.extend(page_urls)
        all_prices.extend(price_list)
        # hacer click en el boton "next"
        # break
        
        try:
            # next_button = driver.find_element(By.XPATH, '//a[@aria-label="Next"]')
            # driver.execute_script("arguments[0].scrollIntoView(true);", next_button)  # Desplaza hasta el botón "Next"
            sleep(2)
            next_button = driver.find_element(By.XPATH, '//a[@aria-label="Next"]')
            driver.execute_script("arguments[0].click();", next_button)  # Forzar clic en el botón "Next" usando JavaScript

            sleep(2)  # Espera a que el desplazamiento se complete
            # next_button.click()  # Haz clic en el botón "Next"
        except NoSuchElementException:
            # print(len(set(all_prices)), set(all_prices))
            break
    driver.quit()
    all_urls = list(set(all_urls))
    all_prices = list(set(all_prices))

    return all_urls, all_prices

In [6]:
url_guest_favorite = "https://www.airbnb.com/rooms/17618950?adults=1&category_tag=Tag%3A8678&enable_m3_private_room=true&photo_id=1258742482&search_mode=regular_search&check_in=2024-10-31&source_impression_id=p3_1727423252_P3uSk5kdl3FQVL7_&previous_page_section_name=1000&federated_search_id=29bac5dc-6e88-4a96-9f0f-fae6c05a3597&guests=1&check_out=2024-11-01"
url_no_guest_favorite = "https://www.airbnb.com/rooms/5413930?adults=1&enable_m3_private_room=true&search_mode=regular_search&check_in=2025-01-05&source_impression_id=p3_1727425011_P3toaJxneSLX1aej&previous_page_section_name=1000&federated_search_id=f573770f-c781-45af-a464-dd123435c95f&guests=1&check_out=2025-01-09"
url_new = "https://www.airbnb.com/rooms/27701970?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-14&check_out=2024-11-19&source_impression_id=p3_1727424233_P3x3d1pTlPM2C8Zb&previous_page_section_name=1000&federated_search_id=b1b3ca1e-4c9f-42f9-9a32-d8fbd3b10aa3"
url_2 = "https://www.airbnb.com/rooms/5413930?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-12-04&check_out=2024-12-09&source_impression_id=p3_1727425011_P3toaJxneSLX1aej&previous_page_section_name=1000&federated_search_id=f573770f-c781-45af-a464-dd123435c95f"
url_problema = "https://www.airbnb.com/rooms/41538482?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=1076432064&search_mode=regular_search&check_in=2024-10-30&check_out=2024-11-04&source_impression_id=p3_1727690290_P34aOaVgGAgAFFpB&previous_page_section_name=1000&federated_search_id=ad80992d-5bc2-4687-b24d-0d1f6c79cb68"
url_3 = "https://www.airbnb.com/rooms/1199915616176105267?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=1955356580&search_mode=regular_search&check_in=2024-11-04&check_out=2024-11-09&source_impression_id=p3_1727690322_P3f_su6KFa0YS6tG&previous_page_section_name=1000&federated_search_id=04476faa-a009-4449-8a1e-e7dfbf28f27a"
url_900_reviews = "https://www.airbnb.com/rooms/14996188?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-09&check_out=2024-11-14&source_impression_id=p3_1727696700_P31z7erJwV7ODI40&previous_page_section_name=1000&federated_search_id=64ab16be-1707-46e7-bce7-955ee4d6bd89"
url_4 = "https://www.airbnb.com/rooms/47476769?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-18&check_out=2024-11-23&source_impression_id=p3_1727697208_P3G-OrMWYXvW785E&previous_page_section_name=1000&federated_search_id=64ab16be-1707-46e7-bce7-955ee4d6bd89"
url_5 = "https://www.airbnb.com/rooms/3712191?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-10-28&check_out=2024-11-02&source_impression_id=p3_1727697705_P3VtOyl8YBojuvt-&previous_page_section_name=1000&federated_search_id=a3da65d1-f655-4932-b203-00d7ea5e1447"
url_no_reviews = "https://www.airbnb.com/rooms/1254652310065298533?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-10-01&check_out=2024-10-06&source_impression_id=p3_1727697624_P3xRvRTXu_LjJG4n&previous_page_section_name=1000&federated_search_id=33e9c87d-3ce4-41b0-9447-3f9b75712890"

otro = "https://www.airbnb.com/rooms/46879110?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-23&check_out=2024-11-28&source_impression_id=p3_1729415435_P3goYUf_EKLmPY5a&previous_page_section_name=1000&federated_search_id=a1ea0a9b-8fcd-44d6-88cf-2a1419162f4c"
otro2 = "https://www.airbnb.com/rooms/1206601525586537688?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-10-09&check_out=2024-10-14&source_impression_id=p3_1728231176_P3p7_QPcWY5ktkLY&previous_page_section_name=1000&federated_search_id=86e37045-ce12-4340-8bc0-bfdbe2294f18"
guest_favorite, rating, number_reviews, type_host, hosting_time, all_reviews, title, complete_data_list, price = scraping_airbnb(otro)

print(f"guest_favorite: {guest_favorite}")
print(f"rating: {rating}")
print(f"number_of_reviews: {number_reviews}")
print(f"type_host: {type_host}")
print(f"hosting_time: {hosting_time}")
print(f"Price: {price}")
print(all_reviews)
print(f"title: {title}")
print(len(all_reviews))
print(f"complete_list: {complete_data_list}")

print(f"guest_favorite: {type(guest_favorite)}")
print(f"rating: {type(rating)}")
print(f"number_of_reviews: {type(number_reviews)}")
print(f"type_host: {type(type_host)}")
print(f"hosting_time: {type(hosting_time)}")
print(f"Price: {type(price)}")
print(type(all_reviews))
# print(len(all_reviews))

guest_favorite: True
rating: Rated 4.95 out of 5 stars.
4.95
number_of_reviews: 155
type_host: 2 guests
hosting_time: Superhost · 4 years hosting
Price: € 65 x 5 nights
['I had to come back and stay again this second time. Clean and Cozy. Excelent attention!', 'Loved the area. She was friendly. Super super clean. I mean the sheets smelled fresh and clean. Spotless floors. Everything accessible both indoors and out. Quiet neighborhood. Everything local within minutes. Planning a future stay for sure. Worth the money!!', '10/10, incredible place/a perfect place!', 'Very good place, all better than the pictures. Really that 100% recommended, kindness, responsibility , cleanliness and more is what awaits them in the place , I would come back a thousand times ☺️', "This was a great place to stay! There's restaurant and stores super close by. It was a cozy place to stay and I would definitely stay there again and might end up staying there again!", 'I loved staying at Madelyn’s place. It had

In [4]:
url_nottingham = 'https://www.airbnb.com/s/Nottingham--England--United-Kingdom/homes'
url_san_francisco = "https://www.airbnb.com/s/San-Francisco--California--United-States/homes"
url_las_vegas = "https://www.airbnb.com/s/las-vegas/homes"
url_chicago = "https://www.airbnb.com/s/Chicago--Illinois--United-States/homes"
url_charlotte = "https://www.airbnb.com/s/Charlotte--North-Carolina--United-States/homes"
url_austin = "https://www.airbnb.com/s/Austin--Texas--United-States/homes"
url_seattle = "https://www.airbnb.com/s/Seattle--Washington--United-States/homes"
url_miami = "https://www.airbnb.com/s/Miami--Florida--United-States/homes"
urls, prices = scraping_urls(url_charlotte)
urls

['https://www.airbnb.com/rooms/650702828700825319?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-05&check_out=2024-11-10&source_impression_id=p3_1729548869_P344DJucHj6IC5id&previous_page_section_name=1000&federated_search_id=1795ce96-5732-448c-a91b-892a478872e9',
 'https://www.airbnb.com/rooms/772364645959328202?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-18&check_out=2024-11-23&source_impression_id=p3_1729548971_P3j0lcCZHACmzHSI&previous_page_section_name=1000&federated_search_id=055db45a-4679-458a-96c5-a7a17f757f0e',
 'https://www.airbnb.com/rooms/22670403?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-03&check_out=2024-11-08&source_impression_id=p3_1729548919_P39ktOGI8QwdZg3o&previous_page_section_name=1000&federated_search_id=b26e5ea0-5053-4ae1-b2e8-55aead47a0cd',
 'https://www.airbnb.com/rooms/1

In [5]:
len(urls)

270

In [6]:
df2 = pd.DataFrame(columns=['title', 'guest_favorite', 'rating', 'number_reviews', 'type_host', 'hosting_time', 'price', 'all_reviews', 'complete_data_list', 'url'])

In [7]:
# df = pd.DataFrame()
n = 0
for url in urls:
    n = n + 1
    # print(f"{n} {price} {url}")
    guest_favorite, rating, number_reviews, type_host, hosting_time, all_reviews, title, complete_data_list, price = scraping_airbnb(url) # price,
    print(f"{n} {price} {url}")
    new_row = pd.DataFrame({
                'title' : [title],
                'guest_favorite': [guest_favorite],
                'rating': [rating],
                'number_reviews': [number_reviews],
                'type_host': [type_host],
                'hosting_time': [hosting_time],
                'price': [price],
                'all_reviews': [all_reviews],
                'complete_data_list' : [complete_data_list],
                'url' : [url]
            })
    df2 = pd.concat([df2, new_row], ignore_index=True)
    # if n > 5:
    #     break

# df.to_csv("SanFrancisco1.csv")
df2

1 € 101 x 5 nights https://www.airbnb.com/rooms/650702828700825319?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-05&check_out=2024-11-10&source_impression_id=p3_1729548869_P344DJucHj6IC5id&previous_page_section_name=1000&federated_search_id=1795ce96-5732-448c-a91b-892a478872e9
2 € 83 x 5 nights https://www.airbnb.com/rooms/772364645959328202?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-18&check_out=2024-11-23&source_impression_id=p3_1729548971_P3j0lcCZHACmzHSI&previous_page_section_name=1000&federated_search_id=055db45a-4679-458a-96c5-a7a17f757f0e
3 nan https://www.airbnb.com/rooms/22670403?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-03&check_out=2024-11-08&source_impression_id=p3_1729548919_P39ktOGI8QwdZg3o&previous_page_section_name=1000&federated_search_id=b26e5ea0-5053-4ae1-b2e8-55aead47a0cd
4

,title,guest_favorite,rating,number_reviews,type_host,hosting_time,price,all_reviews,complete_data_list,url
0,The Cottage at Southpark,False,5.0,5.0 out of 5 stars from 3 reviews\n5.0 · 3 rev...,4 guests,New Host,€ 101 x 5 nights,[],[4 guests · 1 bedroom · 2 beds · 1 bath],https://www.airbnb.com/rooms/65070282870082531...
1,Octopus Garden North End EV studio,True,Rated 4.95 out of 5 stars.\n4.95,142,4 guests,Superhost · 2 years hosting,€ 83 x 5 nights,[We love staying here! Love how the place is a...,[4 guests · 1 bedroom · 1 bed · 1 bath],https://www.airbnb.com/rooms/77236464595932820...
2,Upper Room with Convenient Access + Privacy,True,Rated 4.94 out of 5 stars.\n4.94,253,2 guests,Superhost · 7 years hosting,NaN,[this studio apartment is small but perfect fo...,[2 guests · Studio · 1 bed · 1 bath],https://www.airbnb.com/rooms/22670403?adults=1...
3,3 BR Cozy Lakefront Cottage - LKN - Fire pit,True,Rated 5.0 out of 5 stars.\n5.0,5,10 guests,Superhost · 8 years hosting,€ 140 x 5 nights,[],[10 guests · 3 bedrooms · 5 beds · 2.5 baths],https://www.airbnb.com/rooms/11929096465783864...
4,"Prime Location Retreat: Pool & Parking, Sleeps 6!",False,4.47,17,4 guests,2 years hosting,€ 61 x 5 nights,"[Nice location. Nice place!, Bowen’s place was...",[4 guests · Studio · 2 beds · 1 bath],https://www.airbnb.com/rooms/11674635981062854...
...,...,...,...,...,...,...,...,...,...,...
265,Uptown in 5 minutes,True,Rated 4.89 out of 5 stars.\n4.89,18,1 bed,6 years hosting,€ 37 x 5 nights,[everything was great I misunderstood a few th...,[1 bed · No bathroom],https://www.airbnb.com/rooms/10863252373576693...
266,Quaint Townhouse in CLT,True,Rated 4.88 out of 5 stars.\n4.88,80,6 guests,1 year hosting,€ 92 x 5 nights,[We were in Charlotte for a couple of nights b...,[6 guests · 3 bedrooms · 3 beds · 2.5 baths],https://www.airbnb.com/rooms/73753946842849159...
267,Modern & Stylish 1BR >Dilworth>Restaurants>Shops,True,Rated 4.9 out of 5 stars.\n4.9,390,4 guests,Superhost · 6 years hosting,€ 88 x 5 nights,"[Great location and clean, Great place quite a...",[4 guests · 1 bedroom · 2 beds · 1 bath],https://www.airbnb.com/rooms/40040746?adults=1...
268,Traveler's Hideaway at Lisa's,True,Rated 4.96 out of 5 stars.\n4.96,28,1 queen bed,Superhost · 6 months hosting,NaN,"[Ellisa’s home was beautiful, clean, and an am...",[1 queen bed · Dedicated bathroom],https://www.airbnb.com/rooms/11218337286339931...


In [8]:
df = pd.DataFrame(columns=['title', 'guest_favorite', 'rating', 'number_reviews', 'type_host', 'hosting_time', 'price', 'all_reviews', 'complete_data_list', 'url'])

In [9]:
# df = pd.DataFrame()
n = 0
for url in urls:
    n = n + 1
    # print(f"{n} {price} {url}")
    guest_favorite, rating, number_reviews, type_host, hosting_time, all_reviews, title, complete_data_list, price = scraping_airbnb(url) # price,
    print(f"{n} {price} {url}")
    new_row = pd.DataFrame({
                'title' : [title],
                'guest_favorite': [guest_favorite],
                'rating': [rating],
                'number_reviews': [number_reviews],
                'type_host': [type_host],
                'hosting_time': [hosting_time],
                'price': [price],
                'all_reviews': [all_reviews],
                'complete_data_list' : [complete_data_list],
                'url' : [url]
            })
    df = pd.concat([df, new_row], ignore_index=True)
    # if n > 5:
    #     break

# df.to_csv("SanFrancisco1.csv")
df

1 € 32 x 5 nights https://www.airbnb.com/rooms/25185506?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=499167290&search_mode=regular_search&check_in=2024-11-05&check_out=2024-11-10&source_impression_id=p3_1729520074_P32iCV7IlQE5flji&previous_page_section_name=1000&federated_search_id=8eec2f9e-9695-48e6-b6ab-b77c69c1c6d2
2 € 69 x 5 nights https://www.airbnb.com/rooms/43576117?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-10-26&check_out=2024-10-31&source_impression_id=p3_1729519994_P3aGt8kglKvY5odF&previous_page_section_name=1000&federated_search_id=12602d5c-4420-465f-9558-b95bfc1982b0
3 € 115 x 5 nights https://www.airbnb.com/rooms/50051128?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-10-27&check_out=2024-11-01&source_impression_id=p3_1729520004_P3cdpEMCGvgk2W4T&previous_page_section_name=1000&federated_search_id=356

,title,guest_favorite,rating,number_reviews,type_host,hosting_time,price,all_reviews,complete_data_list,url
0,B2 - Next To Downtown Chicago - Room,False,4.59,291,1 queen bed,7 years hosting,€ 32 x 5 nights,[A good place for my first travel and location...,[1 queen bed · Shared bathroom],https://www.airbnb.com/rooms/25185506?adults=1...
1,Gorgeous Studio 15 Minutes From Ohare!,True,Rated 4.98 out of 5 stars.\n4.98,135,2 guests,Superhost · 3 years hosting,€ 69 x 5 nights,[I'd give a higher than 5 star rating if possi...,[2 guests · Studio · 1 bed · 1 bath],https://www.airbnb.com/rooms/43576117?adults=1...
2,"Steps to Mag Mile, 2 BD , fast Wi-Fi, W&D",True,Rated 4.98 out of 5 stars.\n4.98,208,4 guests,Superhost · 3 years hosting,€ 115 x 5 nights,[Great comfortable place that was very clean. ...,[4 guests · 2 bedrooms · 2 beds · 1 bath],https://www.airbnb.com/rooms/50051128?adults=1...
3,Great Location Studio in the Gold Coast!!,True,Rated 4.89 out of 5 stars.\n4.89,405,2 guests,Superhost · 11 years hosting,€ 80 x 5 nights,[My husband and I had a 5 day stay at Ensavi’s...,[2 guests · 1 bedroom · 1 bed · 1 bath],https://www.airbnb.com/rooms/32406694?adults=1...
4,"Polk Street Coach House Apartment, Little Ital...",True,Rated 4.99 out of 5 stars.\n4.99,818,6 guests,Superhost · 9 years hosting,€ 92 x 5 nights,[Kenneth Airbnb was excellent and I would defi...,[6 guests · 2 bedrooms · 2 beds · 1 bath],https://www.airbnb.com/rooms/8706915?adults=1&...
...,...,...,...,...,...,...,...,...,...,...
250,Vibrant 1BR Apartment close to Dining,False,4.73,40,2 guests,2 years hosting,€ 55 x 5 nights,[Luanne was quick to respond to questions.\nCo...,[2 guests · 1 bedroom · 1 bed · 1 bath],https://www.airbnb.com/rooms/79215779482594926...
251,Best Studio Apt in Chicago close to Boystown,False,4.63,156,2 guests,2 years hosting,€ 71 x 5 nights,"[The place is exactly as described, and the pi...",[2 guests · Studio · 1 bed · 1 bath],https://www.airbnb.com/rooms/66456406916062449...
252,"Spacious Studio in Little Italy, Near West Loop",True,Rated 4.97 out of 5 stars.\n4.97,33,2 guests,Superhost · 5 years hosting,€ 134 x 5 nights,[Really nice place in a great location. Had a ...,[2 guests · 1 bedroom · 1 bed · 1 bath],https://www.airbnb.com/rooms/94354673538173637...
253,Modern Lakefront Retreat with Breathtaking Views!,False,4.73,86,5 guests,Superhost · 5 years hosting,€ 128 x 5 nights,"[Great place, awesome host. We loved the lake ...",[5 guests · 2 bedrooms · 3 beds · 2 baths],https://www.airbnb.com/rooms/53846745?adults=1...


In [10]:
df1 = df.copy()

In [11]:
def get_type_host(hosting_time):
    if isinstance(hosting_time, str) and '·' in hosting_time:
        return hosting_time.split('·')[0].strip()  # Extrae la primera parte antes del "·"
    else:
        return "normal_host"  # Si no hay "·" o no es string, lo marca como "normal host"

# Función para limpiar la columna "hosting_time"
def clean_hosting_time(hosting_time):
    if isinstance(hosting_time, str) and '·' in hosting_time:
        return hosting_time.split('·')[1].strip()  # Toma la parte después del "·"
    else:
        return hosting_time  # Si no hay "·" o no es string, no cambia el valor

# Aplicar las funciones al DataFrame
df['type_host'] = df['hosting_time'].apply(get_type_host)
df['hosting_time'] = df['hosting_time'].apply(clean_hosting_time)

df

,title,guest_favorite,rating,number_reviews,type_host,hosting_time,price,all_reviews,complete_data_list,url
0,B2 - Next To Downtown Chicago - Room,False,4.59,291,normal_host,7 years hosting,€ 32 x 5 nights,[A good place for my first travel and location...,[1 queen bed · Shared bathroom],https://www.airbnb.com/rooms/25185506?adults=1...
1,Gorgeous Studio 15 Minutes From Ohare!,True,Rated 4.98 out of 5 stars.\n4.98,135,Superhost,3 years hosting,€ 69 x 5 nights,[I'd give a higher than 5 star rating if possi...,[2 guests · Studio · 1 bed · 1 bath],https://www.airbnb.com/rooms/43576117?adults=1...
2,"Steps to Mag Mile, 2 BD , fast Wi-Fi, W&D",True,Rated 4.98 out of 5 stars.\n4.98,208,Superhost,3 years hosting,€ 115 x 5 nights,[Great comfortable place that was very clean. ...,[4 guests · 2 bedrooms · 2 beds · 1 bath],https://www.airbnb.com/rooms/50051128?adults=1...
3,Great Location Studio in the Gold Coast!!,True,Rated 4.89 out of 5 stars.\n4.89,405,Superhost,11 years hosting,€ 80 x 5 nights,[My husband and I had a 5 day stay at Ensavi’s...,[2 guests · 1 bedroom · 1 bed · 1 bath],https://www.airbnb.com/rooms/32406694?adults=1...
4,"Polk Street Coach House Apartment, Little Ital...",True,Rated 4.99 out of 5 stars.\n4.99,818,Superhost,9 years hosting,€ 92 x 5 nights,[Kenneth Airbnb was excellent and I would defi...,[6 guests · 2 bedrooms · 2 beds · 1 bath],https://www.airbnb.com/rooms/8706915?adults=1&...
...,...,...,...,...,...,...,...,...,...,...
250,Vibrant 1BR Apartment close to Dining,False,4.73,40,normal_host,2 years hosting,€ 55 x 5 nights,[Luanne was quick to respond to questions.\nCo...,[2 guests · 1 bedroom · 1 bed · 1 bath],https://www.airbnb.com/rooms/79215779482594926...
251,Best Studio Apt in Chicago close to Boystown,False,4.63,156,normal_host,2 years hosting,€ 71 x 5 nights,"[The place is exactly as described, and the pi...",[2 guests · Studio · 1 bed · 1 bath],https://www.airbnb.com/rooms/66456406916062449...
252,"Spacious Studio in Little Italy, Near West Loop",True,Rated 4.97 out of 5 stars.\n4.97,33,Superhost,5 years hosting,€ 134 x 5 nights,[Really nice place in a great location. Had a ...,[2 guests · 1 bedroom · 1 bed · 1 bath],https://www.airbnb.com/rooms/94354673538173637...
253,Modern Lakefront Retreat with Breathtaking Views!,False,4.73,86,Superhost,5 years hosting,€ 128 x 5 nights,"[Great place, awesome host. We loved the lake ...",[5 guests · 2 bedrooms · 3 beds · 2 baths],https://www.airbnb.com/rooms/53846745?adults=1...


In [12]:
df.to_csv("chicago2.csv")

In [13]:
len(df[df["price"] != np.nan])

255

In [8]:
df = df2.copy()

In [9]:
def get_type_host(hosting_time):
    if isinstance(hosting_time, str) and '·' in hosting_time:
        return hosting_time.split('·')[0].strip()  # Extrae la primera parte antes del "·"
    else:
        return "normal_host"  # Si no hay "·" o no es string, lo marca como "normal host"

# Función para limpiar la columna "hosting_time"
def clean_hosting_time(hosting_time):
    if isinstance(hosting_time, str) and '·' in hosting_time:
        return hosting_time.split('·')[1].strip()  # Toma la parte después del "·"
    else:
        return hosting_time  # Si no hay "·" o no es string, no cambia el valor

# Aplicar las funciones al DataFrame
df['type_host'] = df['hosting_time'].apply(get_type_host)
df['hosting_time'] = df['hosting_time'].apply(clean_hosting_time)

df

,title,guest_favorite,rating,number_reviews,type_host,hosting_time,price,all_reviews,complete_data_list,url
0,The Cottage at Southpark,False,5.0,5.0 out of 5 stars from 3 reviews\n5.0 · 3 rev...,normal_host,New Host,€ 101 x 5 nights,[],[4 guests · 1 bedroom · 2 beds · 1 bath],https://www.airbnb.com/rooms/65070282870082531...
1,Octopus Garden North End EV studio,True,Rated 4.95 out of 5 stars.\n4.95,142,Superhost,2 years hosting,€ 83 x 5 nights,[We love staying here! Love how the place is a...,[4 guests · 1 bedroom · 1 bed · 1 bath],https://www.airbnb.com/rooms/77236464595932820...
2,Upper Room with Convenient Access + Privacy,True,Rated 4.94 out of 5 stars.\n4.94,253,Superhost,7 years hosting,NaN,[this studio apartment is small but perfect fo...,[2 guests · Studio · 1 bed · 1 bath],https://www.airbnb.com/rooms/22670403?adults=1...
3,3 BR Cozy Lakefront Cottage - LKN - Fire pit,True,Rated 5.0 out of 5 stars.\n5.0,5,Superhost,8 years hosting,€ 140 x 5 nights,[],[10 guests · 3 bedrooms · 5 beds · 2.5 baths],https://www.airbnb.com/rooms/11929096465783864...
4,"Prime Location Retreat: Pool & Parking, Sleeps 6!",False,4.47,17,normal_host,2 years hosting,€ 61 x 5 nights,"[Nice location. Nice place!, Bowen’s place was...",[4 guests · Studio · 2 beds · 1 bath],https://www.airbnb.com/rooms/11674635981062854...
...,...,...,...,...,...,...,...,...,...,...
265,Uptown in 5 minutes,True,Rated 4.89 out of 5 stars.\n4.89,18,normal_host,6 years hosting,€ 37 x 5 nights,[everything was great I misunderstood a few th...,[1 bed · No bathroom],https://www.airbnb.com/rooms/10863252373576693...
266,Quaint Townhouse in CLT,True,Rated 4.88 out of 5 stars.\n4.88,80,normal_host,1 year hosting,€ 92 x 5 nights,[We were in Charlotte for a couple of nights b...,[6 guests · 3 bedrooms · 3 beds · 2.5 baths],https://www.airbnb.com/rooms/73753946842849159...
267,Modern & Stylish 1BR >Dilworth>Restaurants>Shops,True,Rated 4.9 out of 5 stars.\n4.9,390,Superhost,6 years hosting,€ 88 x 5 nights,"[Great location and clean, Great place quite a...",[4 guests · 1 bedroom · 2 beds · 1 bath],https://www.airbnb.com/rooms/40040746?adults=1...
268,Traveler's Hideaway at Lisa's,True,Rated 4.96 out of 5 stars.\n4.96,28,Superhost,6 months hosting,NaN,"[Ellisa’s home was beautiful, clean, and an am...",[1 queen bed · Dedicated bathroom],https://www.airbnb.com/rooms/11218337286339931...


In [10]:
df.to_csv("charlotte2.csv")